# Actually get predicted grwoth for each merchant

purpose:
- run through predicted growth over all merchants

In [4]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler

In [5]:
# read data
growth = pd.read_csv("../data/curated/final_model/input/fortnightly_agg_merchant_transactions_NOTAKE_NOFRAUD.csv")

In [6]:
# just get last week
growth_final_week = growth[(growth['Year']==2022) & (growth['Fortnight'] == 16)]

In [7]:
# read data
growth_train = pd.read_csv("../data/curated/final_model/input/fortnightly_agg_merchant_transactions_train_NOTAKE_NOFRAUD.csv")

In [8]:
# just get last week
growth_train_final_week = growth[(growth['Year']==2022) & (growth['Fortnight'] == 1)]

In [7]:
def get_Er(growth_final_week):

    # standardise
    growth_standardised = pd.DataFrame()

    missing_abn_list = list()

    for abn in list(growth_final_week['merchant_abn']):

        try:
            with open(f'../models/growth_standardiser/sum_transactions/{abn}.pickle', 'rb') as g:
                SS1 = pickle.load(g)

            with open(f'../models/growth_standardiser/number_of_customers/{abn}.pickle', 'rb') as g:
                SS2 = pickle.load(g)

            with open(f'../models/growth_standardiser/distinct_customers/{abn}.pickle', 'rb') as g:
                SS3 = pickle.load(g)

        
            dta = growth_final_week[growth_final_week['merchant_abn'] == abn]

            dta['sum_transactions_STD'] = SS1.transform(dta[['sum_transactions']])
            dta['number_of_customers_STD'] = SS2.transform(dta[['number_of_customers']])
            dta['distinct_customers_STD'] = SS3.transform(dta[['distinct_customers']])

            growth_standardised = pd.concat([growth_standardised, dta])
        
        except:
            missing_abn_list.append(abn)


    # use validate set to fit standardisers for certain merchants because they were nonexistent in training set
    growth_fill_missing = pd.read_csv("../data/curated/final_model/input/fortnightly_agg_merchant_transactions_validate_NOTAKE_NOFRAUD.csv")


    for id, dta in growth_fill_missing.groupby(['merchant_abn']):
        
        if id in missing_abn_list:
            # fit the StandardScaler using train model
            SS1 = StandardScaler()
            SS1.fit(dta[['sum_transactions']])
            
            SS2 = StandardScaler()
            SS2.fit(dta[['number_of_customers']])

            SS3 = StandardScaler()
            SS3.fit(dta[['distinct_customers']])

            # export the model
            with open(f'../models/growth_standardiser/sum_transactions/{id}.pickle', 'wb') as f:
                pickle.dump(SS1,f)
            with open(f'../models/growth_standardiser/number_of_customers/{id}.pickle', 'wb') as f:
                pickle.dump(SS2,f)
            with open(f'../models/growth_standardiser/distinct_customers/{id}.pickle', 'wb') as f:
                pickle.dump(SS3,f)


    for abn in missing_abn_list:

        with open(f'../models/growth_standardiser/sum_transactions/{abn}.pickle', 'rb') as g:
            SS1 = pickle.load(g)

        with open(f'../models/growth_standardiser/number_of_customers/{abn}.pickle', 'rb') as g:
            SS2 = pickle.load(g)

        with open(f'../models/growth_standardiser/distinct_customers/{abn}.pickle', 'rb') as g:
            SS3 = pickle.load(g)


        dta = growth_final_week[growth_final_week['merchant_abn'] == abn]

        dta['sum_transactions_STD'] = SS1.transform(dta[['sum_transactions']])
        dta['number_of_customers_STD'] = SS2.transform(dta[['number_of_customers']])
        dta['distinct_customers_STD'] = SS3.transform(dta[['distinct_customers']])

        growth_standardised = pd.concat([growth_standardised, dta])


    # drop unstandardised columns
    growth_standardised = growth_standardised.drop(['sum_transactions', 'number_of_customers', 'distinct_customers'], axis=1)
    # rename original columns with t-0
    growth_standardised = growth_standardised.rename(columns={'sum_transactions_STD': 'sum_transactions_STD_t-0', 'number_of_customers_STD': 'number_of_customers_STD_t-0', 'distinct_customers_STD': 'distinct_customers_STD_t-0'})


    with open(f'../models/growth/delta_revenue.pickle', 'rb') as g:
        LR1 = pickle.load(g)

    with open(f'../models/growth/delta_number_of_customers.pickle', 'rb') as g:
        LR2 = pickle.load(g)

    with open(f'../models/growth/delta_distinct_customers.pickle', 'rb') as g:
        LR3 = pickle.load(g)


    # iteratively find growth for future T+t periods
    for i in range(7):
        print(i)
        if i == 0:
            growth_standardised, all_Er = run_one_prediction(growth_standardised, i, LR1, LR2, LR3, all_Er = None)
        
        else:
            growth_standardised, all_Er = run_one_prediction(growth_standardised, i, LR1, LR2, LR3, all_Er = all_Er)

    weighted_avgs = list()

    for row in all_Er.iterrows():

        weighted_avg = sum([row[1][i+1] * (1/(2**i+1)) for i in range(7)])
        
        weighted_avgs.append(weighted_avg)
        
        
    # get final dataframe
    final_Er = all_Er[['merchant_abn']]

    final_Er['weighted E(r)'] = weighted_avgs

    return final_Er

Load the predictors for each of the three categories

In [8]:
def run_one_prediction(growth_standardised, i, LR1, LR2, LR3, all_Er = None):    
    pred1 = LR1.predict(growth_standardised[['sum_transactions_STD_t-0', 'number_of_customers_STD_t-0', 'distinct_customers_STD_t-0']])
    pred2 = LR2.predict(growth_standardised[['sum_transactions_STD_t-0', 'number_of_customers_STD_t-0', 'distinct_customers_STD_t-0']])
    pred3 = LR3.predict(growth_standardised[['sum_transactions_STD_t-0', 'number_of_customers_STD_t-0', 'distinct_customers_STD_t-0']])

    # get the delta values
    growth_standardised['delta(sum_transactions)'] = pred1 
    growth_standardised['delta(number_of_customers)'] = pred2
    growth_standardised['delta(distinct_customers)'] = pred3

    # get the t+1 values
    growth_standardised['sum_transactions_STD_t+1'] = growth_standardised['sum_transactions_STD_t-0'] + growth_standardised['delta(sum_transactions)']
    growth_standardised['number_of_customers_STD_t+1'] = growth_standardised['number_of_customers_STD_t-0'] + growth_standardised['delta(number_of_customers)']
    growth_standardised['distinct_customers_STD_t+1'] = growth_standardised['distinct_customers_STD_t-0'] + growth_standardised['delta(distinct_customers)']

    # de-standardise
    growth_new = pd.DataFrame()

    for abn in list(growth_standardised['merchant_abn']):

        try:
            with open(f'../models/growth_standardiser/sum_transactions/{abn}.pickle', 'rb') as g:
                SS1 = pickle.load(g)

        
            dta = growth_standardised[growth_standardised['merchant_abn'] == abn]

            dta['sum_transactions_t-0'] = SS1.inverse_transform(dta[['sum_transactions_STD_t-0']])

            dta['sum_transactions_t+1'] = SS1.inverse_transform(dta[['sum_transactions_STD_t+1']])

            growth_new = pd.concat([growth_new, dta])
        
        except:
            pass
    
    # get the percentage growth
    growth_new['E(r)'] = (growth_new['sum_transactions_STD_t+1']-growth_new['sum_transactions_STD_t-0'])/growth_new['sum_transactions_STD_t-0']
    # because growth at a minimum can only be -100%
    growth_new['E(r)'] = growth_new['E(r)'].apply(lambda x:x if x >= -1 else -1)

    # if first time running, then create a new dataframe
    if all_Er is None:
        all_Er = growth_new[['merchant_abn']]
    
    all_Er[f'E(r)_{i}'] = growth_new['E(r)']

    # reset growth_standardised and return these files
    growth_standardised = growth_new[['merchant_abn', 'sum_transactions_STD_t+1', 'number_of_customers_STD_t+1', 'distinct_customers_STD_t+1']]
    growth_standardised = growth_standardised.rename(columns = {'sum_transactions_STD_t+1': 'sum_transactions_STD_t-0',
    'number_of_customers_STD_t+1': 'number_of_customers_STD_t-0', 'distinct_customers_STD_t+1': 'distinct_customers_STD_t-0'})

    return growth_standardised, all_Er

In [ ]:
# get training set and final model's predicted future growths
Er_final_model = get_Er(growth_final_week)
Er_train_model = get_Er(growth_train_final_week)

In [10]:
# Export
Er_final_model.to_csv("../data/curated/final_model/input/E(r)_full.csv", index=False)
Er_train_model.to_csv("../data/curated/final_model/input/E(r)_train.csv", index=False)